In [0]:
#Team 2 Notebook First Commit #
#Commit 2
aqi_county_df = spark.read.csv("/Volumes/workspace/2235_team2/aqi/annual_aqi_by_county_2020.csv", header=True, inferSchema=True)
aqi_county_df.show(10)